# Standardization Tools

> Tools for standardization of short read derived gene exspression matrices with long read transcript matrices


In [1]:
#| default_exp Standardization

In [2]:
#| hide
from nbdev.showdoc import *

In [3]:
#| export
import scanpy as sc
def subset_common_cells(dataset1, dataset2):
    return dataset1[dataset1.obs.index.isin(dataset2.obs_names),:]

In [4]:
# short_read_gene_matrix  = sc.read_h5ad("/data/analysis/data_mcandrew/000-sclr-discovair/notebooks/combined_anndata_lr_sr_gene_matrix.h5ad")


In [5]:
# long_read_transcript_matrix = sc.read_h5ad("/data/analysis/data_mcandrew/longreadtools/nbs/isoform_matrix_discovair_union_of_transcripts.h5ad")

In [6]:
# long_read_with_matched_short_read = subset_common_cells(long_read_transcript_matrix, short_read_gene_matrix)

In [7]:
#| hide
import nbdev; nbdev.nbdev_export()